In [1]:
import re, math, codecs, os

In [ ]:
text = []
tokens = 0
texts = 0
with codecs.open('/home/dryzhova/ruwac-filtered_part.out', 'r', encoding='utf-8') as ruwac:
    for line in ruwac:
        if '</text>' in line:
            parse(text)
            texts += 1
            text = []
        else:
            line = line.split()
            text.append(line)
            tokens += 1
print(texts)
print(tokens)

In [4]:
print(texts)
print(tokens)

707958
1120416048


In [2]:
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec

In [5]:
f = 'lemmas.txt'
data = gensim.models.word2vec.LineSentence(f)

In [ ]:
model_cbow = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50)

In [ ]:
model_cbow.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_cbow.wv.save_word2vec_format(model_path, binary=True)

In [3]:
from pprint import pprint

In [ ]:
pprint(model_cbow.wv.most_similar("тоесть", topn=50))

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec

m = "to.bin"
model = Word2Vec.load(m)

In [3]:
info = []
words = []
final = []
with open('sentences.txt') as fh:
    sentences = fh.readlines()
for sentence in sentences:
    sentence.strip()
    sentence = sentence.split()
    count = 0
    for word in sentence:
        info.append(word)
        if len(info) == 3:
            words.append(info)
            info = []
    final.append(words)
    words = []

In [4]:
print(len(final))

1000


In [5]:
m = "to.bin"

In [6]:
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True, unicode_errors='ignore')

In [7]:
sentence_counts = []
for sentence in final:
    left_sum = 0
    right_sum = 0
    count_l = 0
    count_r = 0
    ind = 0
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1
            continue
        if not ind:
            pass
            try:
                count_l += 1
                left_sum += model.wv[word[1]]
            except Exception as e:
                print(e)
        elif ind:
            try:
                count_r += 1
                right_sum += model.wv[word[1]]
            except Exception as e:
                print(e)
    count = (left_sum, right_sum)
    sentence_counts.append(count)

"word 'шибкоприобщать' not in vocabulary"
"word 'летесть' not in vocabulary"
"word 'напароход' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'искушатьтерпение' not in vocabulary"
"word 'сделаетвешивать' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'ямблих' not in vocabulary"
"word 'теургически' not in vocabulary"
"word 'всеголять' not in vocabulary"
"word 'нидругой' not in vocabulary"
"word 'этозвучать' not in vocabulary"
"word 'решенияпроблема' not in vocabulary"
"word 'разныетемпература' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'ниокрый' not in vocabulary"
"word 'тамзаписывать' not in vocabulary"
"word 'бутовичпотребовать' not in vocabulary"
"word 'этотребование' not in vocabulary"
"word 'восновное' not in vocabulary"
"word 'способнаразвиваться' not in vocabulary"
"word 'менееоплачиваемый' not in vocabulary"
"word 'не

In [22]:
len(sentence_counts)

1000

In [8]:
import numpy as np


In [24]:
def cos(vA, vB):
    cos = np.dot(vA, vB) / (np.sqrt(np.dot(vA,vA)) * np.sqrt(np.dot(vB,vB)))
    return cos

In [ ]:
count = 1
sim = []
for i in sentence_counts:
    try:
        sim.append((count, cos(i[0], i[1])))
    except Exception as e:
        sim.append((count, 0))
        print(e)
    count += 1

In [ ]:
sorted_sim = [x for x in sim if x[1].size == 1]
print(len(sorted_sim))
sorted_sim = sorted(sorted_sim, key=lambda x: x[1], reverse=True)

In [ ]:
with open('part_sim.txt', 'w') as fw:
    for i in sorted_sim:
        a, b = i
        fw.write(str(a) + str(b) + '\n')

## Средние векторы по каждому типу значения

In [10]:
# Вектор - сумма векторов слов
context = [sum(i) for i in sentence_counts]

In [12]:
# Скачиваем разметку
with open('Разметка.txt') as fh:
    lines = list(map(lambda l: l.strip(), fh.readlines()))

In [13]:
from collections import Counter

In [14]:
unique = Counter(lines)
unique

Counter({'дискурс': 12,
         'дополнение': 82,
         'интерпретация': 233,
         'коррекция': 40,
         'модальность': 2,
         'обобщение': 51,
         'обр определение': 4,
         'определение': 81,
         'переспрос': 7,
         'переспрос инт': 49,
         'переспрос как': 7,
         'переформулирование': 108,
         'пояснение': 6,
         'пример': 8,
         'следствие': 257,
         'спецификация': 53})

In [42]:
# Находим средние вектора
average = {}
for x, y in zip(lines, context):
    if x not in average.keys():
        average[x] = 0
    average[x] += y
for name, count in unique.items():
    average[name] /= count    

In [16]:
from pymystem3 import Mystem
m = Mystem()

In [31]:
# Это функция для парсинга предложения
def vectorise(item):
    item = re.sub('[Тт]о есть', 'тоесть', item)
    punctuation = (list(""",;:”$%^&*№()_—=+!...?|[]{}\"/<>`~±§«»°1234567890.""")
               + ['- ', ' -', ' - ', '― ', ' ―', ' ― ', " '", "' "])
    item = item.lower()
    for char in punctuation:
        item = item.replace(char, ' ')
    item = item.strip()
    lemmas = m.lemmatize(item)
    vector = 0
    for word in lemmas:
        if word != 'тоедать' and ' ' not in word:
            try:
                vector+=model.wv[word]
            except Exception as e:
                print(e)
    return vector

In [32]:
# Это функция для подсчета близости к типам значения
def find_type(sentence):
    vector = vectorise(sentence)
    types = []
    for key in average.keys():
        types.append((key, cos(average[key], vector)))
    types = sorted(types, reverse=True, key=lambda x: float(x[1]))
    return types

## Попробуем сравнить ожидания и реальность...

In [28]:
sentence = 'И здесь соблюдалась полная демократия, то есть картину отбирал по своему вкусу очередной дежурный '

Это был пример следствия

In [30]:
find_type(sentence)

"word '  ' not in vocabulary"
"word '\n' not in vocabulary"


[('дополнение', 0.5150654),
 ('переформулирование', 0.49012592),
 ('пояснение', 0.48408276),
 ('переспрос инт', 0.47949627),
 ('обобщение', 0.4521315),
 ('спецификация', 0.44709846),
 ('следствие', 0.44098866),
 ('интерпретация', 0.4382135),
 ('коррекция', 0.40017185),
 ('определение', 0.37295192),
 ('обр определение', 0.36476633),
 ('дискурс', 0.36423248),
 ('пример', 0.36344585),
 ('переспрос как', 0.3363337),
 ('переспрос', 0.2817622),
 ('модальность', 0.24421433)]

In [33]:
sentence = 'Очень, очень, очень желает с тобой познакомиться! — Да с какой же стати очень-то? — То есть не то чтобы... видишь, в последнее время, вот как ты заболел, мне часто и много приходилось об тебе поминать... Ну, он слушал... и как узнал, что ты по юридическому и кончить курса не можешь, по обстоятельствам, то сказал: «Как жаль!» Я и заключил... то есть всё это вместе, не одно ведь это; вчера Заметов...'

А вот тут да, дискурсивное + коррекция

In [34]:
find_type(sentence)

"word '\n' not in vocabulary"


[('коррекция', 0.8114302),
 ('переспрос', 0.6984439),
 ('модальность', 0.6544494),
 ('дискурс', 0.6502634),
 ('пояснение', 0.63295966),
 ('переспрос как', 0.5773925),
 ('переспрос инт', 0.5027975),
 ('интерпретация', 0.41054446),
 ('дополнение', 0.40542364),
 ('переформулирование', 0.38285533),
 ('следствие', 0.32364073),
 ('обобщение', 0.27820936),
 ('спецификация', 0.22453292),
 ('обр определение', 0.20840545),
 ('пример', 0.18345429),
 ('определение', 0.13942231)]

In [36]:
sentence = 'Не дальше, как сегодня, то есть в субботу, опять со статьей произошло что-то'

Дополнение

In [37]:
find_type(sentence)

"word '\n' not in vocabulary"


[('коррекция', 0.5498068),
 ('переспрос', 0.41560814),
 ('дискурс', 0.4116518),
 ('модальность', 0.4116348),
 ('пояснение', 0.39525926),
 ('переспрос как', 0.32113042),
 ('переспрос инт', 0.2628887),
 ('интерпретация', 0.22827996),
 ('дополнение', 0.22050104),
 ('переформулирование', 0.18724458),
 ('следствие', 0.14593317),
 ('обобщение', 0.11832918),
 ('спецификация', 0.09761625),
 ('пример', 0.05407716),
 ('обр определение', 0.032646473),
 ('определение', 0.023882343)]

In [38]:
sentence = 'Он вникал в глубину этого сравнения и разбирал, что такое другие и что он сам, в какой степени возможна и справедлива эта параллель и как тяжела обида, нанесенная ему Захаром; наконец, сознательно ли оскорбил его Захар, то есть убежден ли он был, что Илья Ильич всё равно, что «другой», или так это сорвалось у него с языка, без участия головы'

Интерпретация

In [39]:
find_type(sentence)

"word '\n' not in vocabulary"


[('коррекция', 0.6776349),
 ('пояснение', 0.6570716),
 ('переспрос инт', 0.5828377),
 ('дискурс', 0.57037467),
 ('переспрос', 0.5610046),
 ('модальность', 0.560263),
 ('дополнение', 0.5536548),
 ('переформулирование', 0.53951967),
 ('интерпретация', 0.51508194),
 ('переспрос как', 0.46494588),
 ('следствие', 0.46029556),
 ('обобщение', 0.45438346),
 ('спецификация', 0.4056137),
 ('обр определение', 0.36923954),
 ('определение', 0.3363311),
 ('пример', 0.3074932)]

In [40]:
sentence = 'Недоверчив, скептик, циник... надувать любит, то есть не надувать, а дурачить...'

Коррекция

In [41]:
find_type(sentence)

"word '\n' not in vocabulary"


[('коррекция', 0.60621417),
 ('переспрос как', 0.60252357),
 ('переспрос', 0.55185413),
 ('дискурс', 0.4498994),
 ('пояснение', 0.44851378),
 ('модальность', 0.41822103),
 ('переспрос инт', 0.34584057),
 ('дополнение', 0.27641723),
 ('интерпретация', 0.26756608),
 ('переформулирование', 0.24063896),
 ('следствие', 0.20383212),
 ('обобщение', 0.16916816),
 ('спецификация', 0.1300575),
 ('пример', 0.12139918),
 ('обр определение', 0.11391685),
 ('определение', 0.05643766)]

In [43]:
sentence = '― Я вечный двигатель изобрел.  ― То есть как? ― спросил Аркадий Максимович Фетисов. '

Переспрос как

In [44]:
find_type(sentence)

"word '\n' not in vocabulary"


[('переспрос как', 0.47556219),
 ('коррекция', 0.44880027),
 ('переспрос', 0.38952428),
 ('дискурс', 0.33589965),
 ('модальность', 0.31223887),
 ('пояснение', 0.28923923),
 ('переспрос инт', 0.18186472),
 ('дополнение', 0.16484243),
 ('интерпретация', 0.13822787),
 ('переформулирование', 0.0980868),
 ('обобщение', 0.06303097),
 ('следствие', 0.05465932),
 ('спецификация', 0.053451985),
 ('обр определение', 0.004496517),
 ('определение', -0.029287199),
 ('пример', -0.033277873)]

In [45]:
sentence = '― У меня было плохое настроение.   ― То есть?   ― Мне показалось, что все уже было… '

Переспрос

In [46]:
find_type(sentence)

"word '…' not in vocabulary"
"word '\n' not in vocabulary"


[('коррекция', 0.78499514),
 ('модальность', 0.68834275),
 ('переспрос', 0.6589),
 ('дискурс', 0.6487693),
 ('пояснение', 0.60442305),
 ('переспрос как', 0.5358318),
 ('переспрос инт', 0.44571093),
 ('интерпретация', 0.37431946),
 ('дополнение', 0.33254984),
 ('переформулирование', 0.30610272),
 ('следствие', 0.27309316),
 ('обобщение', 0.20490374),
 ('обр определение', 0.15850283),
 ('спецификация', 0.14296141),
 ('пример', 0.12569338),
 ('определение', 0.05054126)]